# Modelo Reto

---


## Instrucciones

<Descripción del Problema>


## Solución


### Importar librerías

Se importan las librerías necesarias para el entorno y el agente.


In [85]:
# Librerías para el agente y modelo
import agentpy as ap
import numpy as np

# Librearías de Visualización y otros
import matplotlib.pyplot as plt
import IPython

import math

plt.style.use("ggplot")


### Definición de las clases

Se define la clase agente `TrafficLight`, la cuál se encarga de simular a los semáforos, y la clase de modelo `TrafficLightModel`, la cuál se encarga de simular el entorno.


In [86]:
class TrafficLight(ap.Agent):
    def setup(self):
        self.state = 0
        self.states = ["green", "yellow", "red"]
        self.updateState()
        self.time = 0
        self.time_limits = self.p.traffic_times
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación
        self.direction = [0, 1]      # Dirección a la que apunta el semáforo

    def act(self):
        self.time += self.step_time
        if self.time >= self.time_limits[self.state]:
            self.time = 0
            self.state = (self.state + 1) % 3
            self.updateState()

    def updateState(self):
        self.current_state = self.states[self.state]


In [87]:
class Car(ap.Agent):
    """ 
        Esta clase define a un auto.
    """

    def setup(self):
        """ Este método se utiliza para inicializar un robot limpiador. """
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación

        self.direction = [1, 0]      # Dirección a la que viaja el auto
        self.speed = 0.0             # Velocidad en metros por segundo
        self.max_speed = 40          # Máxima velocidad en metros por segundo
        self.state = 1               # Car state: 1 = ok, 0 = dead

    def update_position(self):
        """ Este método se utiliza para inicializar la posición del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return

         # Actualiza la posición según la velocidad actual
        self.model.avenue.move_by(self, [self.speed*self.direction[0], self.speed*self.direction[1]])

    def update_speed(self):
        
        """ Este método se utiliza para inicializar la velocidad del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return
        
        # Obten la distancia más pequeña a uno de los autos que vaya en la misma dirección        
        p = self.model.avenue.positions[self]

        min_car_distance = 1000000
        for car in self.model.cars:
            if car != self:
                # Verifica si el carro va en la misma dirección
                dot_p1 = self.direction[0]*car.direction[0] + self.direction[1]*car.direction[1]                
                
                # Verifica si el carro está atrás o adelante
                p2 = self.model.avenue.positions[car]
                dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

                if dot_p1 > 0 and dot_p2 > 0:                     
                    d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)                    
                    
                    if min_car_distance > d:
                        min_car_distance = d
       
        # Obten la distancia al próximo semáforo
        min_trafficLight_distance = 1000000
        trafficLight_state = 0
        for trafficLight in self.model.trafficAgents:

            # Verifica si el semáforo apunta hacia el vehículo
            dot_p1 = trafficLight.direction[0]*self.direction[0] + trafficLight.direction[1]*self.direction[1]
            
            # Verifica si el semáforo está adelante o atrás del vehículo
            p2 = self.model.avenue.positions[trafficLight]
            dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

            if dot_p1 < 0 and dot_p2 > 0:                            
                d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)  
                
                if min_trafficLight_distance > d:
                    min_trafficLight_distance = d
                    trafficLight_state = trafficLight.state
        
        # Actualiza la velocidad del auto
        if min_car_distance < 2:
            self.speed = 0
            self.state = 1

        elif min_car_distance < 20:
              self.speed = np.maximum(self.speed - 200*self.step_time, 0)

        elif min_car_distance < 50:
              self.speed = np.maximum(self.speed - 80*self.step_time, 0)
                
        elif min_trafficLight_distance < 40 and trafficLight_state == 1:
            self.speed = np.minimum(self.speed + 5*self.step_time, self.max_speed)

        elif min_trafficLight_distance < 50 and trafficLight_state == 1:
            self.speed = np.maximum(self.speed - 20*self.step_time, 0)
            
        elif min_trafficLight_distance < 100 and trafficLight_state == 2:
            self.speed = np.maximum(self.speed - 80*self.step_time, 0)

        else:
            self.speed = np.minimum(self.speed + 5*self.step_time, self.max_speed)


In [88]:
class StreetModel(ap.Model):
    def setup(self):
        self.trafficAgents = ap.AgentList(self, self.p.traffic_lights, TrafficLight)
        self.trafficAgents.step_time =  self.p.step_time
        self.trafficAgents.state = ap.AttrIter([2, 0])
        self.trafficAgents.updateState()
        self.trafficAgents[0].direction = [0, 1]
        self.trafficAgents[1].direction = [0, -1]

        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.step_time =  self.p.step_time
        
        c_north = int(self.p.cars/2)
        c_south = self.p.cars - c_north

        for k in range(c_north):
            self.cars[k].direction = [0,1]

        for k in range(c_south):
            self.cars[k+c_north].direction = [0,-1]

        # Inicializa el entorno
        self.avenue = ap.Space(self, shape=[60, self.p.size], torus = True)
                
        # Agrega los semáforos al entorno
        self.avenue.add_agents(self.trafficAgents, random=True)
        self.avenue.move_to(self.trafficAgents[0], [10, self.p.size*0.5 + 5])
        self.avenue.move_to(self.trafficAgents[1], [50, self.p.size*0.5 - 5])

        # Agrega los autos al entorno
        self.avenue.add_agents(self.cars, random=True)
        for k in range(c_north):
            self.avenue.move_to(self.cars[k], [40, 10*(k+1)])
        
        for k in range(c_south):
            self.avenue.move_to(self.cars[k+c_north], [20, self.p.size - (k+1)*10])

    def step(self):
        self.trafficAgents.act()
        self.cars.update_position()
        self.cars.update_speed()

    def update(self):
        states = np.array(self.trafficAgents.state)
        stateNames = np.array(self.trafficAgents.current_state)

        # carPositions = np.array([self.cars.x, self.cars.z])
        # carSpeeds = np.array([self.cars.speed])

        self.record("Traffic Light State", states)
        self.record("Traffic Light State Names", stateNames)
        # self.record("Car Positions", carPositions)
        # self.record("Car Speeds", carSpeeds)

    def end(self):
        pass


### Simulación

Se definen los parámetros, se crea un objeto de la clase `TrafficLightModel`, y se ejecuta la simulación.


In [89]:
parameters = {
    'step_time': 0.1,          # Paso de tiempo
    "traffic_lights": 2,  # Número de semáforos
    "traffic_times": [10, 7, 3],  # Tiempos de cada estado (red, green, yellow)
    "cars": 1,  # Número de autos
    "steps": 100,  # Número de pasos
    "size": 1000,  # Tamaño del entorno
}


In [90]:
model = StreetModel(parameters)
results = model.run()


Completed: 100 steps
Run time: 0:00:00.055744
Simulation finished


In [91]:
results.variables.StreetModel.head(20)


,Traffic Light State,Traffic Light State Names
t,,
0,"[2, 0]","[red, green]"
1,"[2, 0]","[red, green]"
2,"[2, 0]","[red, green]"
3,"[2, 0]","[red, green]"
4,"[2, 0]","[red, green]"
5,"[2, 0]","[red, green]"
6,"[2, 0]","[red, green]"
7,"[2, 0]","[red, green]"
8,"[2, 0]","[red, green]"


In [92]:
def animation_plot_single(m, ax):    
    ax.set_title(f"Avenida t={m.t*m.p.step_time:.2f}")
    
    colors = ["green", "yellow", "red"]
    
    pos_s1 = m.avenue.positions[m.trafficAgents[0]]    
    ax.scatter(*pos_s1, s=20, c=colors[m.trafficAgents[0].state])
    
    pos_s2 = m.avenue.positions[m.trafficAgents[1]]    
    ax.scatter(*pos_s2, s=20, c=colors[m.trafficAgents[1].state])
    
    ax.set_xlim(0, m.avenue.shape[0])
    ax.set_ylim(0, m.avenue.shape[1])    
    
    for car in m.cars:
        pos_c = m.avenue.positions[car]    
        ax.scatter(*pos_c, s=20, c="black")
    
    ax.set_axis_off()
    ax.set_aspect('equal', 'box')
        
def animation_plot(m, p):    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=24)) 

In [93]:
parameters = {
    'step_time': 1 / 24,          # Paso de tiempo
    "traffic_lights": 2,  # Número de semáforos
    "traffic_times": [5, 2, 7],  # Tiempos de cada estado (green, yellow, red)
    "cars": 20,  # Número de autos
    "steps": 300,  # Número de pasos
    "size": 1000,  # Tamaño del entorno
}

In [94]:
model = StreetModel(parameters)
results = model.run()
animation_plot(StreetModel, parameters)

Completed: 300 steps
Run time: 0:00:00.718663
Simulation finished
